In [1]:
import executorch
import torch

## import all module
we will first import all the modules from the package composed_module

## test on resnet

In [2]:
import executorch
import torch
from torch.export import dynamic_dim
from torch.export import Dim
import executorch.exir as exir
from torch._export import capture_pre_autograd_graph
from torch.export import export, ExportedProgram
from executorch.exir import EdgeProgramManager, to_edge
from executorch.exir import ExecutorchBackendConfig, ExecutorchProgramManager
from executorch.exir.passes import MemoryPlanningPass
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from collections import OrderedDict
import numpy as np
import os
import traceback
from tqdm import tqdm
import warnings
# Step 1: Initialize model with the best available weights
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
# model = alexnet(weights=weights)
model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/peter/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 53.9MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
directory_path = './export_model'
os.makedirs(directory_path, exist_ok=True)
directory_path = './model_detail'
os.makedirs(directory_path, exist_ok=True)
directory_path = './input_weight'
os.makedirs(directory_path, exist_ok=True)
directory_path = './golden'
os.makedirs(directory_path, exist_ok=True)
directory_path = './error'
os.makedirs(directory_path, exist_ok=True)

def edge_transform(model: nn.Module, example_args, name: str):
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            os.chdir('/home/peter/projects/lab03/export_model')
            pre_autograd_aten_dialect = capture_pre_autograd_graph(model, (example_args,))
            aten_dialect: ExportedProgram = export(pre_autograd_aten_dialect, (example_args,))
            edge_program: EdgeProgramManager = to_edge(aten_dialect)
            executorch_program: ExecutorchProgramManager = edge_program.to_executorch(
                ExecutorchBackendConfig(
                    passes=[],  # User-defined passes
                    memory_planning_pass=MemoryPlanningPass("greedy")  # Default memory planning pass
                )
            )
            with open(f'{name}.pte', "wb") as file:
                file.write(executorch_program.buffer)
            os.chdir('/home/peter/projects/lab03/input_weight')  # return to origin
    except Exception as e:
        # print('Error:', e)
        os.chdir('/home/peter/projects/lab03/error')
        traceback.print_exc()  # Print stack trace
        with open(f'{name}_error.txt', 'w') as error_file:
            traceback.print_exc(file=error_file)  # Write stack trace to error.txt
        os.chdir('/home/peter/projects/lab03') 

### generate the resnet pte

In [4]:
# create properties
def gen_exec(true_layers):
    layer_idx = 0
    for layer in tqdm(true_layers, desc="Generating Executions"):
        torch._dynamo.reset() # reset the cache
        os.chdir('/home/peter/projects/lab03/input_weight')
        # print(layer)
        class_name = str(layer.__class__).split(".")[-1].split("'")[0]
        layer_idx += 1
        m_key = "%s_%i" % (class_name, layer_idx)
        loaded_input_data = torch.load(f'{m_key}_input.pth')
        temp_model = layer.eval()
        edge_transform(temp_model, loaded_input_data, f'{m_key}')
    os.chdir('/home/peter/projects/lab03')
    

In [5]:
def summary(model, input_size, batch_size=-1, device=torch.device('cpu'), dtypes=None):
    layers_param , layers , result, params_info = summary_string(
        model, input_size, batch_size, device, dtypes)

    return layers_param , layers


def summary_string(model, input_size, batch_size=-1, device=torch.device('cpu'), dtypes=None):
    if dtypes == None:
        dtypes = [torch.FloatTensor]*len(input_size)

    summary_str = ''
    
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split(".")[-1].split("'")[0]
            module_idx = len(summary)
            m_key = "%s_%i" % (class_name, module_idx + 1)
            summary[m_key] = OrderedDict()
            summary[m_key]["input_shape"] = list(input[0].size())
            summary[m_key]["input_shape"][0] = batch_size
            if isinstance(output, (list, tuple)):
                summary[m_key]["output_shape"] = [
                    [-1] + list(o.size())[1:] for o in output
                ]
            else:
                summary[m_key]["output_shape"] = list(output.size())
                summary[m_key]["output_shape"][0] = batch_size

            params = 0
            
            if(module.__module__.startswith('torch.nn.modules')):
                layer_index = len(layers_param)
                l_key = "%s_%i" % (class_name, layer_index + 1)
                layers_param[l_key] = OrderedDict()
                layers_param[l_key]["input_shape"] = list(input[0].size())
                layers_param[l_key]["input_shape"][0] = batch_size
                layers_param[l_key]["output_shape"] = list(output.size())
                layers_param[l_key]["output_shape"][0] = batch_size
                layers.append(module)
        
            if hasattr(module, "weight") and hasattr(module.weight, "size"):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                summary[m_key]["trainable"] = module.weight.requires_grad
            if hasattr(module, "bias") and hasattr(module.bias, "size"):
                params += torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]["nb_params"] = params

        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
        ):
            hooks.append(module.register_forward_hook(hook))

    # multiple inputs to the network
    if isinstance(input_size, tuple):
        input_size = [input_size]

    # batch_size of 2 for batchnorm
    x = [torch.rand(2, *in_size).type(dtype).to(device=device)
         for in_size, dtype in zip(input_size, dtypes)]

    # create properties
    layers = nn.ModuleList()
    layers_param = OrderedDict()
    summary = OrderedDict()
    hooks = []

    # register hook
    model.apply(register_hook)

    # make a forward pass
    # print(x.shape)
    model(*x)

    # remove these hooks
    for h in hooks:
        h.remove()

    summary_str += "-------------------------------------------------------------------------------------" + "\n"
    line_new = "{:>20}  {:>25} {:>25} {:>15}".format(
        "Layer (type)", "Output Shape","Input Shape", "Param #")
    summary_str += line_new + "\n"
    summary_str += "=====================================================================================" + "\n"
    total_params = 0
    total_output = 0
    trainable_params = 0
    for layer in summary:
        # input_shape, output_shape, trainable, nb_params
        line_new = "{:>20}  {:>25} {:>25} {:>15}".format(
            layer,
            str(summary[layer]["output_shape"]),
            str(summary[layer]["input_shape"]),
            "{0:,}".format(summary[layer]["nb_params"]),
            
        )
        total_params += summary[layer]["nb_params"]

        total_output += np.prod(summary[layer]["output_shape"])
        if "trainable" in summary[layer]:
            if summary[layer]["trainable"] == True:
                trainable_params += summary[layer]["nb_params"]
        summary_str += line_new + "\n"

    # assume 4 bytes/number (float on cuda).
    total_input_size = abs(np.prod(sum(input_size, ()))
                           * batch_size * 4. / (1024 ** 2.))
    total_output_size = abs(2. * total_output * 4. /
                            (1024 ** 2.))  # x2 for gradients
    total_params_size = abs(total_params * 4. / (1024 ** 2.))
    total_size = total_params_size + total_output_size + total_input_size
    # return summary
    # print(layers_param)
    # print("hi")
    return layers_param , layers , summary_str, (total_params, trainable_params)

In [6]:
torch._dynamo.config.cache_size_limit = 256
layer_param , true_layers = summary(model, (3, 224, 224),batch_size = 1)
gen_exec(true_layers)

Generating Executions:   1%|▏         | 2/158 [00:00<00:31,  4.92it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_2609/2993195216.py", line 18, in edge_transform
    aten_dialect: ExportedProgram = export(pre_autograd_aten_dialect, (example_args,))
  File "/opt/conda/envs/executorch/lib/python3.10/site-packages/torch/export/__init__.py", line 586, in export
    return export__RC__(f, args, kwargs, dynamic_shapes=dynamic_shapes)
  File "/opt/conda/envs/executorch/lib/python3.10/site-packages/torch/_export/__init__.py", line 95, in export__RC__
    return export(f, args, kwargs)
  File "/opt/conda/envs/executorch/lib/python3.10/site-packages/torch/_export/__init__.py", line 470, in export
    return _export(
  File "/opt/conda/envs/executorch/lib/python3.10/site-packages/torch/_export/__init__.py", line 646, in _export
    gm, graph_signature = aot_export_module(
  File "/opt/conda/envs/executorch/lib/python3.10/site-packages/torch/_functorch/aot_autograd.py", line 4086, in